In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import re
import os 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist





In [4]:
from data_preprocesser import split_raw_csv_review_file

split_raw_csv_review_file(lines_per_file=100000, max_files=20 , join_business=True)

In [5]:
from data_preprocesser import preprocessed_data_path
import ast
from collections import Counter

def get_categories_count_file(filepath):
    df = pd.read_csv(filepath)
    test = []
    for x in df['categories']:
        try:
            x = x.replace(';',',')
        except:
            test.append('')
        else:
            test.append(x)
    df['categories'] = test
    slist = [st for row in df['categories'] for st in row.split(',') if len(row.split(','))<=5]
    return Counter(slist)

def get_categories_count():
    counter = Counter()
    for i in range(20):
        filepath = preprocessed_data_path + 'yelp_academic_dataset_review_split/yelp_academic_dataset_review_{}.csv'.format((i+1)*100000)
        counter = counter + get_categories_count_file(filepath)
    print(counter.most_common(100))

In [6]:
get_categories_count()

[('Restaurants', 869683), ('Food', 189522), ('Nightlife', 121312), ('Bars', 103647), ('American (Traditional)', 100787), ('Breakfast & Brunch', 98606), ('Mexican', 92402), ('American (New)', 91091), ('Beauty & Spas', 82765), ('Pizza', 80697), ('Italian', 79590), ('Shopping', 79182), ('Sandwiches', 70796), ('Japanese', 67635), ('Event Planning & Services', 64376), ('Burgers', 61341), ('Hotels & Travel', 61205), ('Chinese', 61126), ('Automotive', 60637), ('Arts & Entertainment', 60435), ('Sushi Bars', 58058), ('Seafood', 56797), ('Home Services', 54165), ('Coffee & Tea', 53636), ('Fast Food', 48053), ('Health & Medical', 46333), ('Steakhouses', 41854), ('Hotels', 39001), ('Asian Fusion', 38580), ('Active Life', 34623), ('Desserts', 34121), ('Auto Repair', 33447), ('Thai', 33201), ('Local Services', 33178), ('Buffets', 31160), ('Barbeque', 31061), ('Salad', 30957), ('Nail Salons', 30681), ('Bakeries', 29117), ('Cafes', 27635), ('Mediterranean', 26584), ('Hair Salons', 24945), ('Ice Cream 

In [7]:
categories = ['Restaurants', 'Health & Medical', 'Hair Salons', 'Auto Repair']

In [8]:
from data_preprocesser import create_file

business_types = {
    "Restaurants" : pd.DataFrame(), 
    "Health & Medical" : pd.DataFrame(), 
    "Hair Salons" : pd.DataFrame(), 
    "Auto Repair" : pd.DataFrame()
}

threshold = 30000
create_file(preprocessed_data_path + 'categories_{}/'.format(threshold))

for i in range(20):
    print("File " +str(i))
    filepath = preprocessed_data_path + 'yelp_academic_dataset_review_split/yelp_academic_dataset_review_{}.csv'.format((i+1)*100000)
    df = pd.read_csv(filepath, index_col=[0])
    business_types_list = list(business_types.keys())
    for key in business_types_list:
        if business_types[key].shape[0] < threshold:
            other_keys = [x for x in business_types_list if x != key]
            filter_func = lambda s: key in s and not any(x in s for x in other_keys)
            df_temp = df
            df_temp['test'] = df_temp['categories'].apply(filter_func)
            df_temp = df_temp[df_temp['test']]
            df_temp.drop(columns=['test'], inplace=True)
            business_types[key] = pd.concat([business_types[key], df_temp], ignore_index = True)

for key in business_types_list:
    business_types[key] = business_types[key].head(threshold)
    business_types[key].to_csv(preprocessed_data_path + 
                               'categories_{}/yelp_academic_dataset_review_{}.csv'
                               .format(threshold, key), 
                               index=False
                              )
    


File 0


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


File 1
File 2
File 3
File 4
File 5
File 6
File 7
File 8
File 9
File 10
File 11
File 12
File 13
File 14
File 15
File 16
File 17
File 18
File 19
